In [3]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import keras
from keras import metrics
from keras import regularizers
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import Adam, RMSprop
from keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint
from keras.utils import plot_model
from keras.models import load_model
import matplotlib.pyplot as plt


    
    

df = pd.read_excel(r'Copy of Extraction PT4 14x59 din 2016 pana in prezent - analysis S1928 (003).xlsx', sheet_name='TL4 -40 Nm' )
dataset = df.values
X = dataset[1:,18:22]
Y = dataset[1:,14]
X = X.astype('float')
Y = Y.astype('float')


d1 = pd.DataFrame(X)
d2 = pd.DataFrame(Y)
#d1.drop([1,3,5], axis=1, inplace=True)
d2.replace(0., np.nan, inplace=True)
d2.replace(-1., np.nan, inplace=True)
d1[7] = d2[0]
d1.dropna(inplace=True)
# d1.isnull().any() # Verifica daca avem vre-o valoare de tip NaN

X = d1.values[0:,:-1]
Y = d1.values[0:, -1:]


min_max_scaler = preprocessing.MinMaxScaler()
X_scale = min_max_scaler.fit_transform(X)
Y_scaler = preprocessing.MinMaxScaler()
Y_scale = Y_scaler.fit_transform(Y)




X_train, X_test, Y_train, Y_test = train_test_split(X_scale, Y_scale, test_size=0.2)
X_train.shape, Y_train.shape, X_test.shape, Y_test.shape




def basic_model_3(x_size, y_size, x1 , x2 , x3 , x4):
    t_model = Sequential()
    t_model.add(Dense(x1, activation="tanh", kernel_initializer='normal', input_shape=(x_size,)))
    t_model.add(Dropout(0.2))
    t_model.add(Dense(x2, activation="relu", kernel_initializer='normal', 
        kernel_regularizer=regularizers.l1(0.01), bias_regularizer=regularizers.l1(0.01)))
    t_model.add(Dropout(0.1))
    t_model.add(Dense(x3, activation="relu", kernel_initializer='normal', 
        kernel_regularizer=regularizers.l1_l2(0.01), bias_regularizer=regularizers.l1_l2(0.01)))
    t_model.add(Dropout(0.1))
    t_model.add(Dense(x4, activation="relu", kernel_initializer='normal'))
    t_model.add(Dropout(0.0))
    t_model.add(Dense(y_size))
    t_model.compile(
        loss='mean_squared_error',
        optimizer='nadam',
        metrics=[metrics.mae])
    
    return(t_model)

for x1 in range(1,100):
    for x2 in range(1,100):
        for x3 in range(1,100):
             for x4 in range(1,100):
                    
                model = basic_model_3(4,1,x1,x2,x3,x4)
                epochs = 500
                batch_size = 128
                #print('Epochs: ', epochs)
               # print('Batch size: ', batch_size)
                keras_callbacks = [
                # ModelCheckpoint('/tmp/keras_checkpoints/model.{epoch:02d}-{val_loss:.2f}.hdf5', monitor='val_loss', save_best_only=True, verbose=2)
                # ModelCheckpoint('/tmp/keras_checkpoints/model.{epoch:02d}.hdf5', monitor='val_loss', save_best_only=True, verbose=0)
                # TensorBoard(log_dir='/tmp/keras_logs/model_3', histogram_freq=0, write_graph=True, write_images=True, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None),
                EarlyStopping(monitor='val_mean_absolute_error', patience=20, verbose=0)
                ]



                history = model.fit(X_train, Y_train,
                batch_size=batch_size,
                epochs=epochs,
                shuffle=True,
                verbose=0, # Change it to 2, if wished to observe execution
                validation_data=(X_test, Y_test),
                callbacks=keras_callbacks)
                train_score = model.evaluate(X_train, Y_train, verbose=0)
                valid_score = model.evaluate(X_test, Y_test, verbose=0)
                
                def error(x):
                    emax = 1.
                    emin = 1.
                    for i in range(x):
                        p = model.predict(X_train[i].reshape(1, 4))
                        #print(p, Y_train[i], p/float(Y_train[i]))
                        pp = Y_scaler.inverse_transform(p)
                        py = Y_scaler.inverse_transform(Y_train[i].reshape(1,-1))
                        r = p/float(Y_train[i])
                        if r > emax:
                            emax = r
                        if r < emin:
                             emin = r
                        #print(pp,py, r,i)
                    return emin , emax;
            
                emn , ema = error(20)
                print('errmin,errmax,x1,x2,x3,x4:',emn,ema,x1,x2,x3,x4)

#print('Train MAE: ', round(train_score[1], 4), ', Train Loss: ', round(train_score[0], 4)) 
#print('Val MAE: ', round(valid_score[1], 4), ', Val Loss: ', round(valid_score[0], 4))





errmin,errmax,x1,x2,x3,x4: [[0.60519993]] [[5.075427]] 30 10 1 10
errmin,errmax,x1,x2,x3,x4: [[0.59856284]] [[5.019766]] 30 10 1 40
errmin,errmax,x1,x2,x3,x4: [[0.59835017]] [[5.017982]] 30 10 1 70
errmin,errmax,x1,x2,x3,x4: [[0.5933734]] [[4.976246]] 30 10 31 10
errmin,errmax,x1,x2,x3,x4: [[0.58617085]] [[4.9158425]] 30 10 31 40
errmin,errmax,x1,x2,x3,x4: [[0.5894038]] [[4.9429536]] 30 10 31 70
errmin,errmax,x1,x2,x3,x4: [[0.60542756]] [[5.077336]] 30 10 61 10
errmin,errmax,x1,x2,x3,x4: [[0.5757444]] [[4.8284016]] 30 10 61 40
errmin,errmax,x1,x2,x3,x4: [[0.60353774]] [[5.061486]] 30 10 61 70
errmin,errmax,x1,x2,x3,x4: [[0.5930673]] [[4.9736786]] 30 10 91 10
errmin,errmax,x1,x2,x3,x4: [[0.6038794]] [[5.0643525]] 30 10 91 40
errmin,errmax,x1,x2,x3,x4: [[0.56337124]] [[4.724636]] 30 10 91 70
errmin,errmax,x1,x2,x3,x4: [[0.599205]] [[5.0251513]] 30 40 1 10
errmin,errmax,x1,x2,x3,x4: [[0.6016492]] [[5.04565]] 30 40 1 40
errmin,errmax,x1,x2,x3,x4: [[0.59841317]] [[5.018511]] 30 40 1 70


KeyboardInterrupt: 